<h1 style="text-align: center; font-size: 24pt">ICR : Mini-Projet</h1>
<h2 style="text-align: center; font-size: 18pt">Demo</h2>
<h2 style="text-align: center; font-size: 18pt">Loïc Piccot - 21.05.2025</h2>

This notebook aims to demonstrate the core features of the TalkToTheFuture secure messaging application.
It showcases how users can register, authenticate, send encrypted messages to the future, and retrieve them once unlocked.
For a more interactive experience, a command-line interface (CLI) is also available — simply run:
```bash
python main.py
```

In [1]:
from utils.logger import Tracer, print_header
from models import Client, Server
from models.aad import AAD
from datetime import datetime, date
import questionary
import sys

tr = Tracer(trace_level='DEBUG', default_color='magenta')

# Logging/password update

In [2]:
server = Server(name = 'Server')
transmitter = Client(name = 'Transmitter', password = 'password@transmitter')
transmitter.register_on(server)
transmitter.login_on(server)
transmitter.change_password('myNew@password')
transmitter.logout()
transmitter.change_password('Will fail because not logged in')

[DEBUG]: [Transmitter]: Generating keys...
[DEBUG]: [Transmitter]: Request a registration on Server
[INFO]: [Server]: New user Transmitter was added!
[DEBUG]: [Transmitter]: Getting salt from Server
[DEBUG]: [Transmitter]: Regenerating keys ...
[DEBUG]: [Transmitter]: Sending login request to Server
[INFO]: [Server]: User Transmitter is now connected!
[DEBUG]: [Transmitter]: Session started with Server
[DEBUG]: [Transmitter]: Generating keys...
[DEBUG]: [Transmitter]: Updating credentials on Server
[INFO]: [Server]: Keys updated for Transmitter.
[DEBUG]: [Transmitter]: Sending logout request to Server
[INFO]: [Server]: Transmitter has been logged out.
[DEBUG]: [Transmitter]: Generating keys...
[DEBUG]: [Transmitter]: Updating credentials on Server
[ERROR]: [Server]: No active session found for Transmitter
[WARNING]: [Server]: Transmitter must be logged in to update his keys


False

# Messages

## Registration
- Create Alice and Bob Users 
- Register them on a new server

In [3]:
server = Server(name = 'Server')
alice = Client(name = 'Alice', password = 'alicepwd')
bob = Client(name = 'Bob', password = 'bobpwd')

alice.register_on(server)
bob.register_on(server)

[DEBUG]: [Alice]: Generating keys...
[DEBUG]: [Alice]: Request a registration on Server
[INFO]: [Server]: New user Alice was added!
[DEBUG]: [Bob]: Generating keys...
[DEBUG]: [Bob]: Request a registration on Server
[INFO]: [Server]: New user Bob was added!


True

## Alice sends two messages to Bob
- One is in the past -> Bob will be able to open it now
- One is in the future -> Bob won't be able to open it

In [4]:
alice.login_on(server)

alice.send_message(content='Hi Bob!', receiver_name='Bob', unlock_day=date(year=2025, month=5, day=30))
alice.send_message(content='You will be able to read this message later!', receiver_name='Bob', unlock_day=date(year=2060, month=1, day=1))

[DEBUG]: [Alice]: Getting salt from Server
[DEBUG]: [Alice]: Regenerating keys ...
[DEBUG]: [Alice]: Sending login request to Server
[INFO]: [Server]: User Alice is now connected!
[DEBUG]: [Alice]: Session started with Server
[DEBUG]: [Alice]: Getting Bob public key on Server
[DEBUG]: [Alice]: Encrypting and signing message
[DEBUG]: [Alice]: Sending message on Server
[INFO]: [Server]: Message sent to Bob.
[DEBUG]: [Alice]: Getting Bob public key on Server
[DEBUG]: [Alice]: Encrypting and signing message
[DEBUG]: [Alice]: Sending message on Server
[INFO]: [Server]: Message sent to Bob.


True

## Showing messages authenticated data (non crypted)
- Bob Logs in
- Bob reads metadata of messages he received

In [5]:
bob.login_on(server)
messages = bob.get_messages_aad()

tr.sepline(55, char='-')
for id, msg in enumerate(messages):
    tr.colorprint(f"id {id} : {msg}")
tr.sepline(55, char='-')

[DEBUG]: [Bob]: Getting salt from Server
[DEBUG]: [Bob]: Regenerating keys ...
[DEBUG]: [Bob]: Sending login request to Server
[INFO]: [Server]: User Bob is now connected!
[DEBUG]: [Bob]: Session started with Server
[DEBUG]: [Bob]: Requesting message metadata from Server
[DEBUG]: [Server]: Returning Bob's messages
-------------------------------------------------------
id 0 : [From: Alice | To: Bob | Unlock day: 2025-05-30]
id 1 : [From: Alice | To: Bob | Unlock day: 2060-01-01]
-------------------------------------------------------


## Read message in the past
- Bob reads message 0 (available because in the past)
- Bob can download the message without the symmetric encryption key
- Bob can access the encrypted symmetric encryption key
- Bob can append the encrypted symmetric encryption key to the message and fully decrypt it

In [6]:
bob.login_on(server)
plain_text_msg0 = bob.read_message(0)
tr.colorprint(f'Plaintext of received message 0 : {plain_text_msg0}\n')

locked_msg0 = bob.download_future_message(0)
tr.colorprint(f'Crypted received message 0 :{locked_msg0}\n')

key_msg0 = bob.get_msg_enc_sym_key(0)
tr.colorprint(f'Key of message 0 : {key_msg0}\n')

# Appends the key
locked_msg0["enc_sym_key"] = key_msg0
self_decrypted_msg0 = bob.decrypt_message(locked_msg0)
tr.colorprint(f'Plaintext of self decrypted message 0 : {self_decrypted_msg0}')


[DEBUG]: [Bob]: Getting salt from Server
[DEBUG]: [Bob]: Regenerating keys ...
[DEBUG]: [Bob]: Sending login request to Server
[INFO]: [Server]: User Bob is now connected!
[DEBUG]: [Bob]: Session started with Server
[DEBUG]: [Bob]: Requesting full message (id:0) from Server
[DEBUG]: [Server]: Returning message (id:0) with key
[DEBUG]: [Bob]: Decrypting message content
Plaintext of received message 0 : Hi Bob!

[DEBUG]: [Bob]: Downloading future message (id:0) without key
[DEBUG]: [Server]: Returning message (id:0) with key
Crypted received message 0 :{'enc_sym_key': b'\x83\xc9\xa6\xfc=\t\xf7\xdaC\x11\xec\xb64N+\x8b\xd7\x90\xfc\x11\x8e/\xce"$m\x9b\x96\xff\x1b\xc4N\xd76\x88\xbe\xf2\xc1\x99\x1f\x91,I\xeb\xe8\xea\x12o.\xb6\x91\xd9),\x94\xf27\xa1\xf9\xebV\x07\xf7\xf7u\xb2$S\xb4i\x82\x05\x0f\x1a\xe7V\x88\xff\x8c\xab', 'ciphertext': b'\x92\x17P\xd9\xc1}fg_-\xcc\x02\xe1J\xc2\xb2\xbd)\xd6`b\xcf\xc9$\xc3\xc6\x8e~\x8b.\xd1\x85\xf4\x03\xb2K\xb5\xce)\x97C9\x16\xa8\xf9\x8f\xf0', 'aad': b'Alice|Bob\x

## Read message in the future
- Bob can't read message 1 because the unlock day did not happen
- Bob can download the message without the symmetric key. Therefore, he is not able to decrypt the message yet.

In [7]:
plain_text_msg1 = bob.read_message(1)
tr.colorprint(f"Plain text of message 1 is '{plain_text_msg1}' for now\n")
encrypted_msg1 = bob.download_future_message(1)
tr.colorprint(f"Message 1 is {encrypted_msg1}\n")
key_msg1 = bob.get_msg_enc_sym_key(1)
tr.colorprint(f"Enc_sym_key 1 is '{key_msg1}' for now\n")

[DEBUG]: [Bob]: Requesting full message (id:1) from Server
[WARNING]: [Server]: Access to message (id:1) is restricted until 2060-01-01
[ERROR]: [Bob]: Unable to read message (id:1)
Plain text of message 1 is 'None' for now

[DEBUG]: [Bob]: Downloading future message (id:1) without key
[DEBUG]: [Server]: Returning future message (id:1) without key
Message 1 is {'ciphertext': b'@e?\rl\x94\xd3$%\xa5N;\xcf\xfa2\x13\x0f~\xe1\x08&c\xec\x88\x9f\x8d\xb5\xf5\x8e\xdeA\xfb\x11\xd7j\x82\xa3\xf6\xa5Z2f9\x88?d\xaf`\xfd\x91\x9b\xf5c\xb96\x80\x9f\xfc\xc2\xc0\xa3\x8b\xd0|\x84\xbd\xdc\x9f\xb5\x0e\xa9\x0c\xb6G*V\x97\xdaT\x1dt\x88\xdc}', 'aad': b'Alice|Bob\x10\x18!', 'signature': b'\xce\xbb9\xaf\xb0Gy\xdd\x06[\xf0\xa6\xa0\xaf%\xc9*\xa0\xf6\x98s|\xa9\xc9\\o\xa72;\x17\xba\xf0\x1cp\xd1P"\x8c\xb9\xb4Z\xe2_a\x13\x8b\xbf\xeb\xf0a0\xd3A\xc7=S\xba#O\x85\xc4\x85\x93\x02', 'verify_key': b'\xab\xd4z\r\xea\xe4|B\x185I0\xa3p[\x8fL\xb5S\xe2&ON`i\x91\xaa5q\x91~\xca'}

[DEBUG]: [Bob]: Requesting key for message (id:1)
[